In [1]:
# 构建计算图——LSTM模型
#      embedding
#      LSTM
#      fc
#      train_op
# 训练流程代码
# 数据集封装
#      api: next_batch(batch_size)
# 词表封装
#      api: sentence2id(text_sentence): 句子转换id
# 类别封装
#      api: category2id(text_category)

import tensorflow as tf
import os
import sys
import numpy as np
import math

tf.logging.set_verbosity(tf.logging.INFO)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
def get_default_params():
    return tf.contrib.training.HParams(
        num_embedding_size = 16,  
        num_timesteps = 50, # minibatch长度对齐
        num_lstm_nodes = [32, 32],
        num_lstm_layers = 2,
        num_fc_nodes = 32,
        batch_size = 100,
        clip_lstm_grads = 1.0, # lstm梯度
        learning_rate = 0.001,
        num_word_threshold = 10  # 词频阈值
        
#         num_embedding_size = 32,  
#         num_timesteps = 600, # minibatch长度对齐
#         num_lstm_nodes = [64, 64],
#         num_lstm_layers = 2,
#         num_fc_nodes = 64,
#         batch_size = 100,
#         clip_lstm_grads = 1.0, # lstm梯度
#         learning_rate = 0.001,
#         num_word_threshold = 10  # 词频阈值
    )

hps = get_default_params()

# input file
train_file = './data/cnews.train.seg.txt'
val_file = './data/cnews.val.seg.txt'
test_file = './data/cnews.test.seg.txt'
vocab_file = './data/cnews.vocab.txt'
category_file = './data/cnews.category.txt'
output_file = './data/run_text_rnn'

if not os.path.exists(output_file):
    os.mkdir(output_file)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
print(hps.num_word_threshold)

10


In [4]:
class Vocab:
    def __init__(self, filename, num_word_threshold):
        self._word_to_id = {}
        self._unk = -1
        self._num_word_threshold = num_word_threshold
        self._read_dict(filename)
    
    def _read_dict(self, filename):
        with open(filename, 'r') as f:
            lines = f.readlines()
        for line in lines:
            word, frequency = line.strip('\r\n').split('\t')
            frequency = int(frequency)
            if frequency < self._num_word_threshold:
                continue
            idx = len(self._word_to_id) # id递增
            if word == '<UNK>':
                self._unk = idx
            self._word_to_id[word] = idx
    
    def word_to_id(self, word):
        return self._word_to_id.get(word, self._unk)
    
    @property
    def unk(self):
        return self._unk
    
    def size(self):
        return len(self._word_to_id)
        
    def sentence_to_id(self, sentence):
        word_ids = [self.word_to_id(cur_word) for cur_word in sentence.split()]
        return word_ids
    
    
class CategoryDict:
    def __init__(self, filename):
        self._category_to_id = {}
        with open(filename, 'r') as f:
            lines = f.readlines()
        for line in lines:
            category = line.strip('\r\n')
            idx = len(self._category_to_id)
            self._category_to_id[category] = idx
    
    def size(self):
        return len(self._category_to_id)
        
    def category_to_id(self, category):
        if not category in self._category_to_id:
            raise Execption("%s is not in our category list" % category_name);
        return self._category_to_id[category]
        
        
vocab = Vocab(vocab_file, hps.num_word_threshold)
vocab_size = vocab.size()
tf.logging.info('vocab_size: %d' % vocab.size())

# test_str = '的 在 了 是'
# print(vocab.sentence_to_id(test_str))

category_vocab = CategoryDict(category_file)
num_classes = category_vocab.size()
tf.logging.info('num_classes: %d' % num_classes)
test_str = '时尚'
tf.logging.info('label: %s, id: %d' % (test_str, category_vocab.category_to_id(test_str)))

INFO:tensorflow:vocab_size: 77323
INFO:tensorflow:num_classes: 10
INFO:tensorflow:label: 时尚, id: 5


In [5]:
class TextDataset:
    def __init__(self, filename, vocab, category_vocab, num_timesteps):
        self._vocab = vocab
        self._category_vocab = category_vocab
        self._num_timesteps = num_timesteps
        # matrix
        self._inputs = []
        # vector
        self._outputs = []
        self._indicator = 0
        self._parse_file(filename)
        
    def _parse_file(self, filename):
        tf.logging.info('Loading data from %s', filename)
        with open(filename, 'r') as f:
            lines = f.readlines()
        for line in lines:
            label, content = line.strip('\r\n').split('\t')
            id_label = self._category_vocab.category_to_id(label)
            id_words = self._vocab.sentence_to_id(content)
            id_words = id_words[0: self._num_timesteps] # 长的部分做截断
            padding_num = self._num_timesteps - len(id_words)
            id_words = id_words + [self._vocab.unk for i in range(padding_num)]
            self._inputs.append(id_words)
            self._outputs.append(id_label)
        self._inputs = np.asarray(self._inputs, dtype=np.int32)
        self._outputs = np.asarray(self._outputs, dtype=np.int32)
        self._random_shuffle()
        
    def _random_shuffle(self):
        p = np.random.permutation(len(self._inputs))
        self._inputs = self._inputs[p]
        self._outputs = self._outputs[p]
        
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > len(self._inputs):
            self._random_shuffle()
            self._indicator = 0
            end_indicator = batch_size
        if end_indicator > len(self._inputs):
            raise Execption("batch_size: %d is too large" % batch_size)
        
        batch_inputs = self._inputs[self._indicator: end_indicator]
        batch_outputs = self._outputs[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_inputs, batch_outputs
    
train_dataset = TextDataset(train_file, vocab, category_vocab, hps.num_timesteps)
val_dataset = TextDataset(val_file, vocab, category_vocab, hps.num_timesteps)
test_dataset = TextDataset(test_file, vocab, category_vocab, hps.num_timesteps)

print(train_dataset.next_batch(2))
print(val_dataset.next_batch(2))
print(test_dataset.next_batch(2))

INFO:tensorflow:Loading data from ./data/cnews.train.seg.txt
INFO:tensorflow:Loading data from ./data/cnews.val.seg.txt
INFO:tensorflow:Loading data from ./data/cnews.test.seg.txt
(array([[ 1038,  1726,     4, 31985,     2, 25867,    29,    17,   250,
           16,  1038,  1726,     1,     4, 31985,     2, 25867,    29,
            1,  3932,  4726, 47966,   856,   989,     2,   212,   524,
            1,    53,  4150,     5,  2855, 31985,    64,  3261,    33,
         1660,   260,  3089, 49689,     1,  4728,  2224, 29414,     2,
          212,   229,     3,     4, 64592],
       [ 1967,   187, 12690,   789,  9893,  9951,  1272,  1057,  1550,
        10048,  8158, 32616,  2072,    17,    76, 27382,    16,    37,
         1319,  2056,  1967, 32616,  1061,     4, 10048,  2849,  3710,
         1127, 12690,  1272,   789,  9893,     2, 10160,  2056, 76438,
            1,    19,   187,  9893,     2,  1653,     1,  3049,  1272,
          162,  9951,  1057,     5,   635]], dtype=int32), array(

In [6]:
def create_model(hps, vocab_size, num_classes):
    num_timesteps = hps.num_timesteps
    batch_size = hps.batch_size
    
    inputs = tf.placeholder(tf.int32, (batch_size, num_timesteps))
    outputs = tf.placeholder(tf.int32, (batch_size, ))
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    global_step = tf.Variable(tf.zeros([], tf.int64), name='global_step', trainable=False)
    
    embedding_initializer = tf.random_uniform_initializer(-1.0, 1.0)
    with tf.variable_scope('embedding', initializer=embedding_initializer):
        embeddings = tf.get_variable(
            'embedding',
            [vocab_size, hps.num_embedding_size],
            tf.float32)
        # [1, 10, 7] -> [embeddings[1], embeddings[10], embeddings[7]]
        embed_inputs = tf.nn.embedding_lookup(embeddings, inputs)
        
    scale = 1.0 / math.sqrt(hps.num_embedding_size + hps.num_lstm_nodes[-1]) / 3.0
    lstm_init = tf.random_uniform_initializer(-scale, scale)
    with tf.variable_scope('lstm_nn', initializer = lstm_init):
        cells = []
        for i in range(hps.num_lstm_layers):
            cell = tf.contrib.rnn.BasicLSTMCell(
                hps.num_lstm_nodes[i],
                state_is_tuple = True)
            cell = tf.contrib.rnn.DropoutWrapper(
                cell,
                output_keep_prob = keep_prob)
            cells.append(cell)
        cell = tf.contrib.rnn.MultiRNNCell(cells)
        
        initial_state = cell.zero_state(batch_size, tf.float32)
        # rnn_outputs: [batch_size, num_timesteps, lstm_outputs[-1]]
        rnn_outputs, _ = tf.nn.dynamic_rnn(cell, embed_inputs, initial_state = initial_state)
        last = rnn_outputs[:, -1, :]
        
    fc_init = tf.uniform_unit_scaling_initializer(factor=1.0)
    with tf.variable_scope('fc', initializer=fc_init):
        fc1 = tf.layers.dense(last, hps.num_fc_nodes, activation=tf.nn.relu, name='fc1')
        fc1_dropout = tf.contrib.layers.dropout(fc1, keep_prob)
        logits = tf.layers.dense(fc1_dropout,num_classes,name='fc2')
        
    with tf.name_scope('metrics'):
        softmax_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=outputs)
        loss = tf.reduce_mean(softmax_loss)
        # [0, 1, 5, 4, 2] -> argmax: 2
        y_pred = tf.argmax(tf.nn.softmax(logits), 1, output_type=tf.int32)
        correct_pred = tf.equal(outputs, y_pred)
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    with tf.name_scope('train_op'):
        tvars = tf.trainable_variables()
        for var in tvars:
            tf.logging.info('variable name: %s' %(var.name))
        grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), hps.clip_lstm_grads)
        optimizer = tf.train.AdamOptimizer(hps.learning_rate)
        train_op = optimizer.apply_gradients(zip(grads, tvars), global_step = global_step)
        
    return ((inputs, outputs, keep_prob), (loss, accuracy), (train_op, global_step))

placeholders, metrics, others = create_model(hps, vocab_size, num_classes)
inputs, outputs, keep_prob = placeholders
loss, accuracy = metrics
train_op, global_step = others

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Use keras.layers.dense instead.
INFO:tensorflow:variable name: embedding/embedding:0
INFO:tensorflow:variable name: lstm_nn/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0
INFO:tensorflow:variable name: lstm_nn/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0
INFO:tensorflow:variable name: lstm_

In [ ]:
init_op = tf.global_variables_initializer()
train_keep_prob_value = 0.8
test_keep_prob_value = 1.0

num_train_steps = 10000

# Train: 99.7%
# Valid: 92.7%
# Test:  93.2%
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(num_train_steps):
        if i % 150 ==0:
            batch_inputs,batch_labels = test_dataset.next_batch(hps.batch_size)
        else:
            batch_inputs, batch_labels = train_dataset.next_batch(hps.batch_size)
        outputs_val = sess.run([loss, accuracy, train_op, global_step], 
        feed_dict={
            inputs: batch_inputs,
            outputs: batch_labels,
            keep_prob: train_keep_prob_value,
        })
        loss_val, accuracy_val, _, global_step_val = outputs_val
        if global_step_val % 150 == 0:
            tf.logging.info("teststep: %5d,loss: %3.3f,accuracy:%3.5f" % (global_step_val,loss_val,accuracy_val))
        if global_step_val % 20 == 0:
            tf.logging.info("trainstep: %5d,loss: %3.3f,accuracy:%3.5f" % (global_step_val,loss_val,accuracy_val))

INFO:tensorflow:trainstep:    20,loss: 2.294,accuracy:0.13000
INFO:tensorflow:trainstep:    40,loss: 2.272,accuracy:0.14000
INFO:tensorflow:trainstep:    60,loss: 2.223,accuracy:0.12000
INFO:tensorflow:trainstep:    80,loss: 2.260,accuracy:0.15000
INFO:tensorflow:trainstep:   100,loss: 2.235,accuracy:0.18000
INFO:tensorflow:trainstep:   120,loss: 2.259,accuracy:0.10000
INFO:tensorflow:trainstep:   140,loss: 2.171,accuracy:0.20000
INFO:tensorflow:teststep:   150,loss: 2.114,accuracy:0.23000
INFO:tensorflow:trainstep:   160,loss: 2.105,accuracy:0.21000
INFO:tensorflow:trainstep:   180,loss: 1.941,accuracy:0.29000
INFO:tensorflow:trainstep:   200,loss: 1.984,accuracy:0.25000
INFO:tensorflow:trainstep:   220,loss: 1.895,accuracy:0.25000
INFO:tensorflow:trainstep:   240,loss: 2.017,accuracy:0.17000
INFO:tensorflow:trainstep:   260,loss: 1.932,accuracy:0.26000
INFO:tensorflow:trainstep:   280,loss: 1.801,accuracy:0.30000
INFO:tensorflow:teststep:   300,loss: 1.670,accuracy:0.41000
INFO:tenso

INFO:tensorflow:trainstep:  2380,loss: 0.609,accuracy:0.88000
INFO:tensorflow:teststep:  2400,loss: 0.412,accuracy:0.88000
INFO:tensorflow:trainstep:  2400,loss: 0.412,accuracy:0.88000
INFO:tensorflow:trainstep:  2420,loss: 0.484,accuracy:0.86000
INFO:tensorflow:trainstep:  2440,loss: 0.399,accuracy:0.85000
INFO:tensorflow:trainstep:  2460,loss: 0.379,accuracy:0.84000
INFO:tensorflow:trainstep:  2480,loss: 0.588,accuracy:0.80000
INFO:tensorflow:trainstep:  2500,loss: 0.435,accuracy:0.87000
INFO:tensorflow:trainstep:  2520,loss: 0.337,accuracy:0.89000
INFO:tensorflow:trainstep:  2540,loss: 0.400,accuracy:0.86000
INFO:tensorflow:teststep:  2550,loss: 0.384,accuracy:0.87000
INFO:tensorflow:trainstep:  2560,loss: 0.389,accuracy:0.91000
INFO:tensorflow:trainstep:  2580,loss: 0.366,accuracy:0.90000
INFO:tensorflow:trainstep:  2600,loss: 0.465,accuracy:0.84000
INFO:tensorflow:trainstep:  2620,loss: 0.458,accuracy:0.84000
INFO:tensorflow:trainstep:  2640,loss: 0.254,accuracy:0.92000
INFO:tenso

INFO:tensorflow:trainstep:  4720,loss: 0.061,accuracy:0.98000
INFO:tensorflow:trainstep:  4740,loss: 0.227,accuracy:0.93000
INFO:tensorflow:trainstep:  4760,loss: 0.172,accuracy:0.95000
INFO:tensorflow:trainstep:  4780,loss: 0.135,accuracy:0.95000
INFO:tensorflow:teststep:  4800,loss: 0.218,accuracy:0.97000
INFO:tensorflow:trainstep:  4800,loss: 0.218,accuracy:0.97000
INFO:tensorflow:trainstep:  4820,loss: 0.245,accuracy:0.93000
INFO:tensorflow:trainstep:  4840,loss: 0.174,accuracy:0.97000
INFO:tensorflow:trainstep:  4860,loss: 0.103,accuracy:0.95000
INFO:tensorflow:trainstep:  4880,loss: 0.117,accuracy:0.97000
INFO:tensorflow:trainstep:  4900,loss: 0.083,accuracy:0.98000
INFO:tensorflow:trainstep:  4920,loss: 0.114,accuracy:0.95000
INFO:tensorflow:trainstep:  4940,loss: 0.119,accuracy:0.96000
INFO:tensorflow:teststep:  4950,loss: 0.104,accuracy:0.97000
INFO:tensorflow:trainstep:  4960,loss: 0.179,accuracy:0.96000
INFO:tensorflow:trainstep:  4980,loss: 0.148,accuracy:0.96000
INFO:tenso

INFO:tensorflow:trainstep:  7060,loss: 0.055,accuracy:0.98000
INFO:tensorflow:trainstep:  7080,loss: 0.015,accuracy:0.99000
INFO:tensorflow:trainstep:  7100,loss: 0.110,accuracy:0.97000
INFO:tensorflow:trainstep:  7120,loss: 0.067,accuracy:0.98000
INFO:tensorflow:trainstep:  7140,loss: 0.034,accuracy:0.99000
INFO:tensorflow:trainstep:  7160,loss: 0.102,accuracy:0.98000
INFO:tensorflow:trainstep:  7180,loss: 0.035,accuracy:0.99000
INFO:tensorflow:teststep:  7200,loss: 0.020,accuracy:1.00000
INFO:tensorflow:trainstep:  7200,loss: 0.020,accuracy:1.00000
INFO:tensorflow:trainstep:  7220,loss: 0.053,accuracy:0.99000
INFO:tensorflow:trainstep:  7240,loss: 0.039,accuracy:0.99000
INFO:tensorflow:trainstep:  7260,loss: 0.008,accuracy:1.00000
INFO:tensorflow:trainstep:  7280,loss: 0.239,accuracy:0.97000
INFO:tensorflow:trainstep:  7300,loss: 0.009,accuracy:1.00000
INFO:tensorflow:trainstep:  7320,loss: 0.075,accuracy:0.99000
INFO:tensorflow:trainstep:  7340,loss: 0.113,accuracy:0.98000
INFO:tens